In [1]:
#imports
import numpy as np
from scipy.stats import chisquare
from scipy.stats import poisson
from sklearn.preprocessing import MinMaxScaler
import math
from sklearn.mixture import GaussianMixture # For Expectation maximisation algorithm

In [2]:
# Load Data
data = np.genfromtxt('scale-d-10d.csv', delimiter=' ')
data = data[:,0:10]

labels = np.genfromtxt('scale-d-10d.csv', delimiter=' ',dtype="|U5")
labels = labels[:,10]

In [3]:
# MinMaxScaler did not work properly!
# https://datascience.stackexchange.com/questions/38004/minmaxscaler-returned-values-greater-than-one
def rescale(data, new_min=0, new_max=1):
    """Rescale the data to be within the range [new_min, new_max]"""
    return (data - data.min()) / (data.max() - data.min()) * (new_max - new_min) + new_min

In [4]:
data = rescale(data)

In [5]:
#dependencies: numpy, scipy (for scipy.stats.chisquare)
class P3C:
    
    #class variables
    _alpha = 0.001 #alpha for chi-squared-test

    #Poisson threshold is the only parameter for P3C 
    def __init__(self, poisson_threshold): 
        
        #sklearn-like attributes
        self.labels_ = None
        self.cluster_centers_ = None #"cluster cores"
        
        #internally used variables
        self._poisson_threshold = poisson_threshold
        self._support_set = []
        self._supports = []
        self._approx_proj = []   #nested list: 1st level: attributes, 2nd level:
                                 #different intervals, 3rd level: start and end of inteval.
                                 #used as interface between part 3.1 and 3.2
    
    
    # Methods for 3.1: Projections of true p-signatures (Mahdi and Robert)
    
    # Extra
    def __compute_support(self, M):
        '''Computes Supports of each bin
        This function computes support set of each interval S and its support.
        then assigns the values to self._support_set = [] and self._supports = []
        SupportSet(S)= {x ∈ D | x.aj ∈ S }
        Support(S) = |SupportSet(S)|


        Parameters
        ----------
        self, 

        M : numpy.array 

        Returns
        -------

        '''
        n = M.shape[0] # n = number of data objects
        attribute_number = M.shape[1] # number of attributes 
        bin_number = int(1 + math.log(n,2)) # number of bins


        for i in range(attribute_number):
            # support set of each interval S for attribiute i
            supp_set = [[]for i in range(bin_number)]

            interval_length = 1 / bin_number

            # calculate in which bin should the point be placed based on attribute i
            for j in range(n):
                supp_set_index = math.floor(M[j,i]/interval_length)

                if supp_set_index == len(supp_set):
                    supp_set_index-=1 

                supp_set[supp_set_index].append(M[j,:])

            self._support_set.append(supp_set)
            

        self._supports = [[] for i in range(len(self._support_set))]
        for i in range(len(self._support_set)):
            for j in range(len(self._support_set[i])):
                self._supports[i].append(len(self._support_set[i][j]))

        return 
    
    def __approximate_projection(self):
        
        bins = np.zeros((len(self._supports), len(self._supports[0])), dtype=int)
        for attr_number in range(len(self._supports)): #loop over all attributes
                
            # part 1: create bin array            
            supp = self._supports[attr_number].copy() #make a copy of the supports of the current attribute
            while self.__uniformity_test(supp) == False: #if not uniform find highest element
                max_index = supp.index(max(supp))
                supp.pop(max_index) #remove highest element from list
                
                i = 1 
                while i <= max_index: #loop to adjust max_index according to previousely deleted elements
                    max_index += bins[attr_number, i]
                    i += 1
                bins[attr_number, max_index]  = 1 #mark highest bin
            
            #part 2: create _approx_proj list from bin array
            interval_list = [] #2d list for current attribute
            interval = [] #current interval
            open_interval = False
            
            for i in range(len(bins[attr_number])):
                if open_interval == False: #open new interval
                    if bins[attr_number, i] == 1:
                        interval.append(i)
                        open_interval = True
                if open_interval == True: #close current interval
                    if bins[attr_number, i] == 0:
                        interval.append(i)
                        interval_list.append (interval)
                        interval = []
                        open_interval = False
                    if (i == len(bins[attr_number])-1) and (bins[attr_number, i] == 1): #last bin marked 1
                        interval.append (len(bins[attr_number]))
                        interval_list.append (interval)                
        
            self._approx_proj.append (interval_list)
        
    def __uniformity_test(self, attr):
        if chisquare(attr)[0] < self._alpha:
            #print ("uniform")
            return True
        #print ("non-uniform")
        return False  
    
    # Methods for 3.3: Cluster Cores (Akshey and Jonas)
    
    def __convert_approx_proj_to_dict(_approx_proj):
        ''' Converts _approx_proj from 3.2 to a dictonary
        This is necessary to compute cluster cores with apriori

        Parameters
        ----------
        _approx_proj : three nested lists (# attribut/# interval/ start and end of interval)

        Returns
        -------
        _approx_proj_sig : list of dictonaries, each dictonary is a projection e.g {0: [0.1, 0.2]}

        '''
        _approx_proj_sig = []
        for attribute, row in enumerate(_approx_proj):
            for interval in row:
                _approx_proj_sig.append({attribute:interval})
                
        return _approx_proj_sig
    
    
     def __compute_support_sig(p_signature, dataset):
        '''Computes support for p-signature
        This function computes the support by removing data points
        that do not lie in any of the intervals of the given p-signature

        Parameters
        ----------
        p_signature : dictronary e.g. {0:[0,0.1], 3:[0.1,0.2]} -> Intervals for attributes 0 and 3

        dataset : numpy.ndarray 

        Returns
        -------
        data.shape[0] : number of points in p-signature

        '''

        data = np.copy(dataset)
        for attribute in p_signature:
            interval = p_signature[attribute]
            remove = []
            for i, point in enumerate(data):
                if  interval[0] > point[attribute] or point[attribute] > interval[1]:
                    remove.append(i)
            data = np.delete(data, remove, 0)

        return data.shape[0]

    
    def __compute_exp_support(p_signature, interval, data):
        ''' Computes expected support for a p-signature

        Parameters
        ----------
        p-signature : dictronary e.g. {0:[0,0.1], 3:[0.1,0.2]} -> Intervals for attributes 0 and 3

        interval : list with start and end value of interval

        data : normalized np.ndarray

        Returns
        -------
        support * width : 

        '''

        support = __compute_support_sig(p_signature, data)
        width = abs(interval[0] - interval[1])

        return support*width
    
    
    def __diff_interval(p_signature, pplus1_signature):
        '''Helper function to compute difference in interval for two p-signatures.
           Used for possion threshold

        Parameters
        ---------- 
        p_signature : dictronary e.g. {0:[0,0.1], 3:[0.1,0.2]} -> Intervals for attributes 0 and 3

        pplus1_signature : dictronary e.g. {0:[0,0.1], 3:[0.1,0.2]} -> Intervals for attributes 0 and 3

        Returns
        -------
        interval : 

        '''

        diff = list(set(pplus1_signature) - set(p_signature))
        interval = pplus1_signature[diff[0]]
        
        return interval



     def __check_core_condition(p_signature, pplus1_signature, dataset, threshold=1e-20):
        ''' Checks if probability is smaller than possion threshold: 
        Possion(Supp(k+1 signature), ESupp(k+1 signature)) < possion_threshold
        Returns True is poisson value is smaller than threshold

        and

        Checks if support is larger than expected support: 
        Supp(k+1 signature) > ESupp(k+1 siganature)
        ESupp = Supp(S) * width(S')

        Parameters
        ----------
        p-signatue : dictronary e.g. {0:[0,0.1], 3:[0.1,0.2]} -> Intervals for attributes 0 and 3

        pplus1_signature : dictronary e.g. {0:[0,0.1], 3:[0.1,0.2]} -> Intervals for attributes 0 and 3

        dataset : numpy.ndarray

        threshold : poisson_threshold -> defined by user. default: 1e-20

        Returns
        -------
        true/false : 

        '''
        
        interval = __diff_interval(p_signature, pplus1_signature)
        support = __compute_support_sig(pplus1_signature, dataset)
        expected_support = __compute_exp_support(pplus1_signature, interval, dataset)
        base_condition = support > expected_support
        if base_condition:
            poisson_value = poisson.pmf(support, expected_support) 
            if poisson_value < threshold:
                return True
            else:
                return False
            
            
    def __merge(dict1, dict2):
        '''Helper function to merge to p-signature dictonaries 

        Parameters
        ----------
        dict1: p-signature dictonary

        dict2: p-signature dictronary 

        Returns
        -------
        res : merged p-signature containing of dict1 and dict 2

        '''
        res = {**dict1, **dict2}
        return res

    
    def __a_is_subset_of_b(a,b):
        '''Helper function that checks is dictionary a is a subset of dictionary b 

        Parameters
        ----------
        a : dictionary

        b : dictionary

        Returns
        -------
        bool : truth value

        '''
        return all((k in b and b[k]==v) for k,v in a.items())
    
    
    def __apriori_cores(_approx_proj_sig, data):
        ''' Computes cluster cores in apriori fashion. 
        The function computes maximal p-signatures that fulfill 
        two conditions. 

        Parameters
        ----------
        _approx_proj_sig : list of dictonaries

        data: dataset np.ndarray


        Returns
        -------
        cluster_centers_ : list of p-signatures e.g. [{0: [0.1, 0.2]}, {1: [0.5, 0.8], 2: [0.1, 0.4], 3: [0.5, 0.7]}]

        '''

        # Loop through attributes and intervals (ignore same dimensions)
        _cluster_cores = [_approx_proj_sig]

        while _cluster_cores[-1] != []:
            p_sig_list = []
            for p_sig in _cluster_cores[-1]:
                for one_sig in _approx_proj_sig:
                  ### The second criterion is to avoid double counting
                  ### In case this second criteria gives errors we could make a set of the p-signatures before checking the core condition
                    if list(one_sig.keys())[0] not in p_sig.keys() and list(one_sig.keys())[0] > max(list(p_sig.keys())): 
                        pplus1_sig = __merge(p_sig, one_sig)
                        ### Check core condition
                        if __check_core_condition(p_sig, pplus1_sig, data):
                            p_sig_list.append(pplus1_sig)
            _cluster_cores.append(p_sig_list)
        _cluster_cores.pop()

        # Finds only the unique cluster cores since the above algorithm might be return the same core multiple times 
        cluster_centers_ = []
        for p_sig_list in _cluster_cores:
            for p_sig in p_sig_list:
                if p_sig not in cluster_centers_:
                    cluster_centers_.append(p_sig)

        maximal_cluster_centers_ = cluster_centers_.copy()
        # Check condition 2 for each signature (pruning to maximal cluster cores)
        for cluster in reversed(cluster_centers_):
            for sub_cluster in reversed(cluster_centers_):
                if cluster != sub_cluster: 
                    if __a_is_subset_of_b(sub_cluster, cluster):
                        if sub_cluster in maximal_cluster_centers_:
                            maximal_cluster_centers_.remove(sub_cluster)


        return maximal_cluster_centers_

    def __compute_core_support(cores_p_signatures, data):
        ''' Computes the support for each cluster core. This is necessary for 3.3:
        computing the projected clusters 

        Parameters
        ----------
        cores_p_signatures : list of core signatures e.g. [{0: [0.1, 0.2]}, {1: [0.5, 0.8], 2: [0.1, 0.4], 3: [0.5, 0.7]}]

        Returns
        -------
        cores_support_sets : list of supports for each cluster core

        '''
        cores_support_sets = []
        for p_sig in cores_p_signatures:
            cores_support_sets.append(__compute_support_sig(p_sig, data))
        
        return cores_support_sets
    
     # Methods for 3.3: Computing projected clusters (Manju)
    def __fuzzy_membership_matrix(cluster_core_i,data): 
        '''
        Refines the cluster cores into projected clusters
        Parameters
        ----------
        cluster_core_i:
        
        data:
        
        Returns
        -------
        fuzzy_membership_matrix:
        '''
        fuzzy_membership_matrix=[]
        for i in range(1,n):
            for l in range(1,k):
                if (i in data and l in support_set(cluster_core_i)):
                    if(i not in support_set(cluster_core_i)): 
                        fuzzy_membership_matrix=0
                       # unassigned_datapoints=cluster_core_i.append(i)
                    elif (i in support_set(cluster_core_i)):
                        fuzzy_membership_matrix=(1/support_set(cluster_core_i))[data]
        
        return fuzzy_membership_matrix   
    
                
    def __probability_of_datapoint(fuzzy_membership_matrix,max_iterations):
        '''
          For each data point compute the probability of belonging to each projected cluster using Expectation 
          Maximization(EM)algorithm.
          Parameters
          ----------
          fuzzy_membership_matrix:
          
          max_iterations:
          
          Returns
          -------
          probability_matrix:
        
        '''
        # initialise EM with fuzzy_membership_matrix.cluster members have shorter mahalanobis distances to cluster
        # means than non-cluster members
        max_iterations=10
        gaussian_mixture = GaussianMixture(n_components=2, covariance_type='full').fit(fuzzy_membership_matrix)
        gaussian_mixture.means_
        gaussian_mixture.fit()
        label=gaussian_mixture.predict(fuzzy_membership_matrix)
        
        return gaussian_mixture
        


    #data X is numpy.ndarray with samples x features (no label!)  
    def fit (self, X):
        #all the method calls of 3.1 and 3.2 we have to implement go here...
        self.__compute_support(X)
        self.__approximate_projection()
        
        #self.cluster_center_ = ... #used as interface between part 3.2. and 3.3
        
    #data X is numpy.ndarray with samples x features (no label!)  
    def predict (self, X):
        pass #remove pass when implementing predict (X)
        
        #all the method calls of 3.3, 3.4 and 3.5 we have to implement go here...
        
        #self.labels_ = #final result of the algorithm.
        
    #data X is numpy.ndarray with samples x features (no label!)  
    def fit_predict (self, X):
        self.fit (X)
        self.predict (X)

In [6]:
p3c = P3C (10)
p3c.fit (data)
print (p3c._approx_proj)

[[[0, 14]], [[0, 14]], [[0, 1], [2, 14]], [[0, 1], [2, 14]], [[0, 14]], [[0, 14]], [[0, 14]], [[2, 14]], [[0, 1], [2, 14]], [[0, 1], [2, 14]]]


/Users/mahdyfalah/opt/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:6125: RuntimeWarning: invalid value encountered in true_divide
  terms = (f_obs.astype(np.float64) - f_exp)**2 / f_exp
/Users/mahdyfalah/opt/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:6118: RuntimeWarning: Mean of empty slice.
  f_exp = f_obs.mean(axis=axis, keepdims=True)
